# CoordConv (Coordinate Convolution) — Tam Anlatım + PyTorch Uygulaması

Bu notebook’ta **CoordConv** fikrini sıfırdan, adım adım öğreneceğiz ve PyTorch’ta **temiz bir implementasyon** yazacağız.

## Hedef
- CoordConv **neden ortaya çıktı**?
- Normal conv neden bazen koordinatı "öğrenmekte" zorlanır?
- CoordConv katmanı **ne ekler** (x/y/opsiyonel radius kanalları)?
- `CoordConv2d` nasıl yazılır?
- Küçük bir modelde nasıl kullanılır?

Not: CoordConv’un yaptığı şey basit:
> Modele, feature map’in **mutlak konum bilgisini** ek kanallar olarak verir.


In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(0)
print('torch:', torch.__version__)


torch: 2.9.1+cpu


---
## 1) Normal Conv’un Konum Problemi (neden CoordConv?)

Normal `Conv2d` aynı kernel’i görüntünün her yerinde uygular. Bu özellik çoğu işte iyidir.
Ancak bazı görevlerde modelin **mutlak konumu** bilmesi gerekir:
- "Bu piksel görüntünün neresinde?" (sol üst mü, sağ alt mı?)
- "Objenin merkezi hangi koordinatta?"
- Bounding box / keypoint gibi koordinat regresyon görevleri

Normal conv, mutlak koordinatı doğrudan görmez; konumu dolaylı yollardan öğrenmek zorunda kalır.
CoordConv: konumu öğrenmeye zorlamak yerine **konumu verir**.


---
## 2) CoordConv Ne Yapar?

Input feature map: `x` (B, C, H, W)

CoordConv şunu yapar:
1) Ek koordinat kanalları üretir:
   - `x_coord`: [-1,1] W boyunca
   - `y_coord`: [-1,1] H boyunca
   - (opsiyonel) `r`: merkezden uzaklık
2) Concat eder: (B, C+2, H, W) veya (B, C+3, H, W)
3) Normal `Conv2d` uygular

Yani: **"coord ekle + conv"**


---
## 3) Koordinat Kanalları Üretimi

Koordinatlar genelde normalize edilir:
- x: [-1,1]
- y: [-1,1]

Böylece farklı çözünürlüklerde de ölçek tutarlı kalır.


In [3]:
def make_coord_channels(B, H, W, device, dtype, with_r=False):
    y = torch.linspace(-1.0, 1.0, steps=H, device=device, dtype=dtype)
    x = torch.linspace(-1.0, 1.0, steps=W, device=device, dtype=dtype)
    yy, xx = torch.meshgrid(y, x, indexing='ij')  # (H,W)

    xx = xx[None, None].repeat(B, 1, 1, 1)  # (B,1,H,W)
    yy = yy[None, None].repeat(B, 1, 1, 1)  # (B,1,H,W)

    if with_r:
        rr = torch.sqrt(xx**2 + yy**2)
        return torch.cat([xx, yy, rr], dim=1)
    return torch.cat([xx, yy], dim=1)

B, H, W = 2, 4, 6
coords2 = make_coord_channels(B, H, W, device=torch.device('cpu'), dtype=torch.float32, with_r=False)
coords3 = make_coord_channels(B, H, W, device=torch.device('cpu'), dtype=torch.float32, with_r=True)
print('coords2:', coords2.shape)
print('coords3:', coords3.shape)


coords2: torch.Size([2, 2, 4, 6])
coords3: torch.Size([2, 3, 4, 6])


---
## 4) CoordConv2d Katmanı

`CoordConv2d`:
- Koordinat kanalı üretir
- Girişe concat eder
- Normal conv uygular

Önemli: Conv’un `in_channels` değeri artık `cin + 2` (veya `cin + 3`).


In [4]:
class CoordConv2d(nn.Module):
    def __init__(self, cin, cout, k=3, stride=1, padding=1, with_r=False, bias=False):
        super().__init__()
        self.with_r = with_r
        extra = 3 if with_r else 2
        self.conv = nn.Conv2d(cin + extra, cout, kernel_size=k, stride=stride, padding=padding, bias=bias)

    def forward(self, x):
        B, C, H, W = x.shape
        coords = make_coord_channels(B, H, W, device=x.device, dtype=x.dtype, with_r=self.with_r)
        x = torch.cat([x, coords], dim=1)
        return self.conv(x)

x = torch.randn(2, 8, 32, 32)
layer = CoordConv2d(cin=8, cout=16, k=3, padding=1, with_r=True)
y = layer(x)
print('x:', x.shape)
print('y:', y.shape)


x: torch.Size([2, 8, 32, 32])
y: torch.Size([2, 16, 32, 32])


---
## 5) CoordConvBlock (CoordConv + BN + ReLU)


In [7]:
class CoordConvBlock(nn.Module):
    def __init__(self, cin, cout, k=3, stride=1, padding=1, with_r=False):
        super().__init__()
        self.cc = CoordConv2d(cin, cout, k=k, stride=stride, padding=padding, with_r=with_r, bias=False)
        self.bn = nn.BatchNorm2d(cout)
        self.act = nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.cc(x)
        x = self.bn(x)
        x = self.act(x)
        return x


---
## 6) Mini Model: CoordNet

Bu model stem kısmında CoordConv kullanıyor.
Gerçek projede CoordConv’u genelde erken katmanlarda veya koordinat-regresyon head’lerinde görürsün.


In [8]:
class CoordNet(nn.Module):
    def __init__(self, in_channels=3, num_classes=10, with_r=True):
        super().__init__()
        self.stem = nn.Sequential(
            CoordConvBlock(in_channels, 32, k=3, stride=1, padding=1, with_r=with_r),
            nn.MaxPool2d(2),
        )
        self.stage1 = nn.Sequential(
            nn.Conv2d(32, 64, 3, padding=1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
        )
        self.stage2 = nn.Sequential(
            nn.Conv2d(64, 128, 3, padding=1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
        )
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(128, num_classes)

    def forward(self, x, verbose=False):
        if verbose: print('input:', x.shape)
        x = self.stem(x)
        if verbose: print('stem :', x.shape)
        x = self.stage1(x)
        if verbose: print('s1   :', x.shape)
        x = self.stage2(x)
        if verbose: print('s2   :', x.shape)
        x = self.pool(x)
        x = torch.flatten(x, 1)
        out = self.fc(x)
        if verbose: print('out  :', out.shape)
        return out

model = CoordNet(in_channels=3, num_classes=10, with_r=True)
x = torch.randn(2, 3, 64, 64)
y = model(x, verbose=True)
print('final:', y.shape)


input: torch.Size([2, 3, 64, 64])
stem : torch.Size([2, 32, 32, 32])
s1   : torch.Size([2, 64, 16, 16])
s2   : torch.Size([2, 128, 8, 8])
out  : torch.Size([2, 10])
final: torch.Size([2, 10])


---
## 7) Özet

- CoordConv = koordinat kanalı ekleyip normal conv yapmak
- Modelin mutlak konumu "öğrenmesini" kolaylaştırır
- Özellikle localization / regression görevlerinde faydalıdır
